<a href="https://colab.research.google.com/github/ainfanzon/Cockroach_IAM_Workshop/blob/main/GCP_Colab_notebooks/Exercise_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<img src="https://drive.google.com/uc?id=1XYr9Tyrz31a5kZdo601xD1QWz_YM8-H3">

### CockroachDB is a distributed SQL database that is __*highly scalable*__, __*resilient*__, and __*easy to use*__.

# Identity and Access Management Workshop.
---
## Scalability and Resiliency.

In this section we are going to scale-up the cluster. You will:

1. Add three more nodes to cluster.
1. Create and populate the Northwine database.
1. Simulate two nodes failure.
1. Restart nodes (Optional)
<br>

<html>
<head>
<style>
table, th, td {
  border: 1px solid black;
  border-collapse: collapse;
}
</style>
</head>
<body>

<table style="width:100%">
  <tr>
       <td align="right">
          <img src="https://drive.google.com/uc?id=1VS1jCK6UAUeqdNrKot3BbOZrbMVo2m5l" width="850"
          height="350">
      </td>
  </tr>
</table>

</body>
</html>

---

## 1. Add three more nodes to cluster.

CockroachDB is designed to be low touch and highly automated for operators, while remaining easy to reason about for developers.

**Benefits**

CockroachDB's horizontal scalability allows users to start small and scale out as needed. It also maintains ACID guarantees, so users don't have to risk their data to improve performance.

Execute the next two cells below to overview the current status of the cluster (alternatively use the DB Console). In the cell below, replace the host value with your PUBLIC IP address.

In [1]:
import psycopg2
import pandas as pd

from IPython.display import IFrame, display, HTML, Markdown

pd.set_option('display.max_colwidth', None)

try:
    conn = psycopg2.connect(
        database = 'defaultdb'
      , user = 'root'
      , host = '34.221.252.230'      # Use the PUBLIC IP
      , port = '26257'
      , sslmode = 'disable'
    )
    display(Markdown("### Connection successful!"))
    cursor = conn.cursor()
except psycopg2.OperationalError as e:
    print(f"Error connecting to database: {e}")

### Connection successful!

Display the information of the cluster before adding more data.

In [2]:
cursor.execute("""
SELECT gn.node_id AS "Node ID"
     , gn.advertise_sql_address AS "Advertised Address"
     , gn.build_tag AS "Version"
     , current_timestamp() AT TIME ZONE 'UTC' - gn.started_at AS "Up Time"
     , "ranges" AS "Ranges"
     , leases AS "Leaders"
     , CASE WHEN is_live THEN 'LIVE' ELSE 'DEAD' END AS "status"
     , gl.membership
FROM crdb_internal.gossip_nodes AS gn join crdb_internal.gossip_liveness AS gl USING(node_id)
""")
result_set = cursor.fetchall()
df_result_set = pd.DataFrame(result_set, columns=[desc[0] for desc in cursor.description])
df_result_set.set_index('Node ID', inplace=True)
df_result_set

,Advertised Address,Version,Up Time,Ranges,Leaders,status,membership
Node ID,,,,,,,
1,10.14.0.156:26257,v24.2.3,0 days 00:14:50.616058,54,11,LIVE,active
2,10.14.0.156:26259,v24.2.3,0 days 00:14:50.113539,54,11,LIVE,active
3,10.14.0.156:26258,v24.2.3,0 days 00:14:49.961841,0,0,DEAD,decommissioned
4,10.14.0.156:26260,v24.2.3,0 days 00:14:31.142997,0,0,DEAD,decommissioned
5,10.14.0.156:26261,v24.2.3,0 days 00:14:30.250582,0,0,DEAD,decommissioned
6,10.14.0.156:26262,v24.2.3,0 days 00:14:29.368587,53,10,LIVE,active
7,10.14.0.156:26258,v24.2.3,0 days 00:11:34.809233,53,11,LIVE,active
8,10.14.0.156:26261,v24.2.3,0 days 00:11:34.806642,54,11,LIVE,active


To scale the cluster follow the steps below:

- Use the __**cockroach start**__ command to provision additional nodes to the cluster.

> <p>cockroach start<br>
&emsp;&emsp;--insecure<br>
&emsp;&emsp;--listen-addr=&lt;ip address&gt;:&lt;sql listening port&gt;<br>
&emsp;&emsp;--join=&lt;ip address&gt;:&lt;sql listening port&gt;, ... ,&lt;ip address&gt;:&lt;sql listening port&gt;<br>
&emsp;&emsp;--http-addr=&lt;ip address&gt;:&lt;http listening port&gt;<br>
&emsp;&emsp;--locality=region=us-west,zone=us-west-1a<br> &emsp;&emsp;--store=/home/cockroach/data/cr_data_1<br>
&emsp;&emsp;--background<br>
</p>

Execute the __**add_nodes.sh**__ shell script in the scripts directory.

> </code>
$ /home/cockroach/scripts/add_nodes.sh us-west
</code>

Verify there are six instances of the `cockroach` process running on different ports.

- List all active `cockroach` processes.

> `pgrep -a cockroach | awk '{ print $5}'`

&emsp;&emsp;Each process will be listening on the same IP but different port.

> <code>
--listen-addr=10.0.1.2:26257<br>
--listen-addr=10.0.1.2:26258<br>
--listen-addr=10.0.1.2:26259<br>
--listen-addr=10.0.1.2:26260<br>
--listen-addr=10.0.1.2:26261<br>
--listen-addr=10.0.1.2:26262<br>
</code>

### Few points to note.

Using the DB Console overview.

- Expnd the nodes to see how they are grouped?

<br><br>

---

## 2. Create and populate the Northwind database.
<html>
<head>
<style>
table, th, td {
  border: 1px solid black;
  border-collapse: collapse;
}
</style>
</head>
<body>

<table style="width:100%">
  <tr>
      <td align="right">
          <img src="https://drive.google.com/uc?id=1eM0otn7ieCvBMXVQ0WmXgRKUf41GaS9h" width="850"
          height="650">
      </td>
  </tr>
</table>

</body>
</html>

Execute the following steps to create and load the data to the database:

- Run the sed command below to update the script IP address:

> ```sed -E -i s/HOST_IP/$(hostname -I | awk '{print $1}')/ /home/cockroach/sql/northwind.sql```

&emsp;&emsp;NOTE: Make sure the HTTP server is running on port 3000. ( `python -m http.server 3000` )


> ```cockroach sql --host $(hostname -I) -u root -d default -f /home/cockroach/sql/northwind.sql --insecure```

- On the cockroach **DB Console**:

> <code>
http://IP Address:8080/#/overview/list
</code>


In [5]:
try:
    conn = psycopg2.connect(
        database = 'northwind'
      , user = 'root'
      , host = '34.221.252.230'     # Use the PUBLIC IP
      , port = '26257'
      , sslmode = 'disable'
    )
    display(Markdown("### Connection successful!"))
    cursor = conn.cursor()
except psycopg2.OperationalError as e:
    print(f"Error connecting to database: {e}")

cursor.execute("""
SELECT gn.node_id AS "Node ID"
     , gn.advertise_sql_address AS "Advertised Address"
     , gn.build_tag AS "Version"
     , current_timestamp() AT TIME ZONE 'UTC' - gn.started_at AS "Up Time"
     , "ranges" AS "Ranges"
     , leases AS "Leaders"
     , CASE WHEN is_live THEN 'LIVE' ELSE 'DEAD' END AS "status"
     , gl.membership
FROM crdb_internal.gossip_nodes AS gn join crdb_internal.gossip_liveness AS gl USING(node_id)
""")
result_set = cursor.fetchall()
df_result_set = pd.DataFrame(result_set, columns=[desc[0] for desc in cursor.description])
df_result_set.set_index('Node ID', inplace=True)
df_result_set

### Connection successful!

,Advertised Address,Version,Up Time,Ranges,Leaders,status,membership
Node ID,,,,,,,
1,10.14.0.156:26257,v24.2.3,0 days 00:24:05.147166,125,47,LIVE,active
2,10.14.0.156:26259,v24.2.3,0 days 00:24:04.644647,90,20,LIVE,active
3,10.14.0.156:26258,v24.2.3,0 days 00:24:04.492949,0,0,DEAD,decommissioned
4,10.14.0.156:26260,v24.2.3,0 days 00:23:45.674105,0,0,DEAD,decommissioned
5,10.14.0.156:26261,v24.2.3,0 days 00:23:44.781690,0,0,DEAD,decommissioned
6,10.14.0.156:26262,v24.2.3,0 days 00:23:43.899695,88,19,LIVE,active
7,10.14.0.156:26258,v24.2.3,0 days 00:20:49.340341,87,19,LIVE,active
8,10.14.0.156:26261,v24.2.3,0 days 00:20:49.337750,91,20,LIVE,active


#### Compare the output with the earlier result.

- How many nodes are there in the cluster?
- Is the number of ranges higher or lower than before?
- **In the DB Console what is the number of replicas different than the sql output? why?**

---

## 3. Simulate failure of two nodes on different regions.

<html>
<head>
<style>
table, th, td {
  border: 1px solid black;
  border-collapse: collapse;
}
</style>
</head>
<body>

<table style="width:100%">
  <tr>
      <td align="right">
          <img src="https://drive.google.com/uc?id=1MCqaWMMCNDr2dYaP_JMDZykiQxpu_8Gd" width="850" height="375">
      </td>
  </tr>
</table>

</body>
</html>

- Set the __**time_until_store_dead**__ to reduce the amount of time the cluster waits before considering a node dead. The default is five minutes and the minimum allowed is 1 minute and 15 seconds. Execute the command below on the second terminal.

> <code>
echo $(hostname -I) | xargs -I {} cockroach sql --insecure --host={}:26257 -d defaultdb --execute="SET CLUSTER SETTING server.time_until_store_dead = '1m15s';"
</code>

- To simulate an availability zone failure that brings down two nodes in the cluster execute the `az_failure.sh` script.

>```
$ su - roachie
$ /home/cockroach/scripts/az_failure.sh
```

- Check the DB Console after a couple of minutes :

    - How many nodes are reported dead?
    - In the replication status now many under replicated ranges are there?

- Check if the Northwind database is still operational after a whole availability zone went down.

In [6]:
cursor.execute("""
SELECT p.product_name AS "Product Name"
     , SUM(od.unit_price * CAST(od.quantity AS FLOAT) * (1.0 - od.discount)) AS Sales
FROM products AS p INNER JOIN order_details AS od ON od.product_id = p.product_id
GROUP BY p.product_name
ORDER BY Sales DESC LIMIT 5
""")
result_set = cursor.fetchall()
df_result_set = pd.DataFrame(result_set, columns=[desc[0] for desc in cursor.description])
df_result_set.set_index('Product Name', inplace=True)
df_result_set

,sales
Product Name,
Côte de Blaye,141396.735
Thüringer Rostbratwurst,80368.672
Raclette Courdavault,71155.700
Tarte au sucre,47234.970
Camembert Pierrot,46825.480


---

## 4. Restart nodes (Optional).

Execute the `restore_nodes.sh` script that was generated in the previous step.

<code>
./restore_nodes.sh
</code>

- What is happening with the under replicated ranges?
- Are all the nodes operational again?

NOTE:

If there are still dead nodes you can decommision them using the following command:

```cockroach node decommission &lt;node id&gt; --insecure --host=&lr<Private IP Address&gt>

---
## CockroachDB is a distributed SQL database that is __*highly scalable*__, __*resilient*__, and __*easy to use*__.
<img src="https://drive.google.com/uc?id=1XYr9Tyrz31a5kZdo601xD1QWz_YM8-H3">

---

# Appendix

Workshop CRDB user id and passowrd

> <p>uid = roachie<br>
pwd = roachfan
</p>

List CRDB process id and process name.

> <code>pgrep -l cockroach</code>

List the listening address of each `cockroach` process.

> <code>pgrep -a cockroach | awk '{ print $5}'</code>

Kill ALL CRDB processes

> <code>kill -9  $(pgrep cockroach)</code>

Remove all CRDB files

> <code>sudo rm -fR /home/cockroach/data/*</code>

Update the IP in the script

> <code>sed -E -i s/HOST_IP/$(hostname -I | awk '{print $1}')/ northwind.sql</code>